## data preperation

### file opening

In [1]:
import numpy as np
from implementations import *
from proj1_helpers import *
from datetime import datetime
from nan_predic import *
np.random.seed(2)
import pandas as pd

SyntaxError: invalid syntax (nan_predic.py, line 35)

In [2]:
y,X,ids = load_csv_data("train.csv")

In [3]:
X = nan_predic(X)

In [4]:
X = np.where(X == -999., np.nan, X)
y_1 = y[~np.isnan(X).any(axis=1)]
X_1 = X[~np.isnan(X).any(axis=1)]
y_2 = y[np.isnan(X).any(axis=1)]
X_2 = X[np.isnan(X).any(axis=1)]

In [5]:
def poly_features(X):
    X_copy = np.copy(X)
    for i in range(X_copy.shape[1]):
        for j in range(i,X_copy.shape[1]):
            X = np.column_stack((X, X_copy[:,i]*X_copy[:,j]))
    return X

def make_features(X):
    # converting -999. to nan to use np.nanmean and np.nanstd
    X = np.where(X == -999., np.nan, X)
    # standardizing the data Xd = (X_d - E[X_d])/(std(X_d))
    X, means, stds = standardize(X)
    # since data is standirdized, the mean is more or less 0 for each feature so replacing by zero is reasonable and helps computations
    X = np.where(np.isnan(X), 0, X)
    # adding the 1 padding
    return np.column_stack((np.ones(X.shape[0]), X))

In [6]:
def preproc(X):
    col_means = np.nanmean(X, axis=0)
    col_means = np.nanmedian(X, axis=0)
    idxs = np.where(np.isnan(X))
    X[idxs] = np.take(col_means, idxs[1])

    #feature 1: correlations der_mass_MMC
    X_gt_mmc = np.array(X[:,0], copy=True)
    # X_0_cop = np.array(X[:,0], copy=True)
    X_gt_mmc[X_gt_mmc <= 140] = 140
    # X = np.column_stack((X, X_gt_mmc))
    X[:,0][X[:,0] > 140] = 140
    X = np.column_stack((X, X_gt_mmc))

    #feature 2: add momentums
    #tau momentum
    tau_px = X[:,13]*np.cos(X[:,15])
    tau_py = X[:,13]*np.sin(X[:,15])
    tau_pz = X[:,13]*np.sinh(X[:,14])
    tau_mod = X[:,13]*np.cosh(X[:,14])
    X = np.column_stack((X, tau_px,tau_py,tau_pz))
    #lep momentum
    lep_px = X[:,16]*np.cos(X[:,18])
    lep_py = X[:,16]*np.sin(X[:,18])
    lep_pz = X[:,16]*np.sinh(X[:,17])
    lep_mod = X[:,16]*np.cosh(X[:,17])
    X = np.column_stack((X, lep_px,lep_py,lep_pz))
    #leading jet momentum
    jet_px = X[:,23]*np.cos(X[:,25])
    jet_py = X[:,23]*np.sin(X[:,25])
    jet_pz = X[:,23]*np.sinh(X[:,24])
    jet_mod = X[:,23]*np.cosh(X[:,24])
    X = np.column_stack((X, jet_px,jet_py,jet_pz))
    #subleading jet momentum
    subjet_px = X[:,26]*np.cos(X[:,28])
    subjet_py = X[:,26]*np.sin(X[:,28])
    subjet_pz = X[:,26]*np.sinh(X[:,27])
    subjet_mod = X[:,26]*np.cosh(X[:,27])
    X = np.column_stack((X, subjet_px,subjet_py,subjet_pz))

    #feature 8: total invariant mass
    term_1 = np.sqrt(tau_px**2 + tau_py**2 + tau_pz**2) + np.sqrt(lep_px**2 + lep_py**2 + lep_pz**2) \
    + np.sqrt(jet_px**2 + jet_py**2 + jet_pz**2) + np.sqrt(subjet_px**2 + subjet_py**2 + subjet_pz**2)
    term_2 = (tau_px + lep_px + jet_px + subjet_px)**2 + (tau_py + lep_py + jet_py + subjet_py)**2 \
            + (tau_pz + lep_pz + jet_pz + subjet_pz)**2
    inv_mass = np.sqrt(term_1**2 - term_2)


    #feature 3: abs angles
    #der_met_phi_centrality
    X[:,11] = np.abs(X[:,11])
    #tau phi
    X[:,15] = np.abs(X[:,15])
    #lep phi
    X[:,18] = np.abs(X[:,18])
    #met phi
    X[:,20] = np.abs(X[:,20])
    #lead jet phi
    X[:,24] = np.abs(X[:,24])
    #sublead jet phi
    X[:,27] = np.abs(X[:,27])
    #R sep abs
    X[:,7] = np.abs(X[:,7])

    #feature 9: log
    inv_log_cols = (0,1,2,3,4,5,7,8,9,10,12,13,16,19,21,23,26)
    X_inv_log_cols = np.log(1 / (1 + X[:, inv_log_cols]))
    X = np.hstack((X, X_inv_log_cols))
    # X_test_inv_log_cols = np.log(1 / (1 + X_test[:, inv_log_cols]))
    # X_test = np.hstack((X_test, X_test_inv_log_cols))


    #feature 4: categorical PRI_jet_num
    jet_num_0 = (X[:,22] == 0).astype(int)
    jet_num_1 = (X[:,22] == 1).astype(int)
    jet_num_2 = (X[:,22] == 2).astype(int)
    jet_num_3 = (X[:,22] == 3).astype(int)

    # #feature 5: pt ratios
    # #tau_lep_ratio = PRI_tau_pt/PRI_lep_pt
    tau_lep_ratio = X[:,13]/X[:,16]
    # #met_tot_ratio = PRI_met/PRI_met_sumet
    met_tot_ratio = X[:,19]/X[:,21]
    # X = np.column_stack((X, tau_lep_ratio,jets_ratio,met_tot_ratio))
    X = np.column_stack((X, tau_lep_ratio,met_tot_ratio))

    # #feature 6: jets_diff_angle
    jets_diff_angle = np.cos(X[:,24]-X[:,27])
    X = np.column_stack((X, jets_diff_angle))

#     X = np.column_stack((X, inv_mass))
    X = make_features(X)
    X = np.column_stack((X, jet_num_0, jet_num_1, jet_num_2, jet_num_3))
    return X

In [7]:
X_1 = preproc(X_1)
X_2 = preproc(X_2)

C:\Users\leose\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in log
C:\Users\leose\Desktop\ML_project\implementations.py:18: RuntimeWarning: invalid value encountered in true_divide
  x = x / std_x
C:\Users\leose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  
C:\Users\leose\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:959: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
C:\Users\leose\Desktop\ML_project\implementations.py:15: RuntimeWarning: Mean of empty slice
  mean_x = np.nanmean(x, axis = 0)
C:\Users\leose\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [11]:
# np.random.shuffle(X)
cutoff_1 = int(0.8*((X_1.shape)[0]))
cutoff_2 = int(0.8*((X_2.shape)[0]))
X_1_train = X_1#[:cutoff_1]
y_1_train = y_1#[:cutoff_1]
X_1_test = X_1[cutoff_1:]
y_1_test = y_1[cutoff_1:]
X_2_train = X_2#[:cutoff_2]
y_2_train = y_2#[:cutoff_2]
X_2_test = X_2[cutoff_2:]
y_2_test = y_2[cutoff_2:]

# Least Squares

## gradient descent Least Squares

In [12]:
max_iter = 100
losses = []
ws = []
gammas = np.linspace(0.01,0.03,21)
for gamma in gammas:
    w, loss = least_squares_GD(y_1_train, X_1_train, np.zeros(X_1_train.shape[1]), max_iter, gamma)
    losses.append(loss)
    ws.append(w)

In [13]:
index = np.argmin(losses)
w = ws[index]
loss = losses[index]
gammas[index]

0.03

In [14]:
pred_tr = X_1_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_train - pred_tr)) / X_1_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

accuracy on training set is of 74.72399341715138%, loss is of 0.17375074752477893


In [15]:
pred = X_1_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred)) / X_1_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

accuracy on test set is of 74.72849623559198 %


## stochastic gradient descent least squares

In [16]:
max_iter = 20000
losses = []
ws = []
gammas = np.linspace(0.01,0.03,21)
for gamma in gammas:
    w, loss = least_squares_SGD(y_1_train, X_1_train, np.zeros(X_1_train.shape[1]), max_iter, gamma)
    losses.append(loss)
    ws.append(w)

In [17]:
index = np.argmin(losses)
w = ws[index]
loss = losses[index]
gammas[index]

0.01

In [18]:
pred_tr = X_1_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_train - pred_tr)) / X_1_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

accuracy on training set is of 66.45612211583948%, loss is of 0.2622736619593481


In [19]:
pred = X_1_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred)) / X_1_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

accuracy on test set is of 66.45012992204677 %


## least squares

In [ ]:
w, loss = least_squares(y_1_train, X_1_train)

In [ ]:
pred_tr = X_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_train - pred_tr)) / X_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

In [ ]:
pred = X_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_test - pred)) / X_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

## ridge regression

In [20]:
lambdas = np.linspace(0.01, 0.1, 11)
losses = []
ws = []
for lambda_ in lambdas:
    w, loss = ridge_regression(y_1_train, X_1_train, lambda_)
    ws.append(w)
    losses.append(loss)

In [21]:
accuracies = []
for w in ws:
    pred_test = X_1_test @ w
    pred_test = np.where(pred_test > 1/2, 1, 0)
    accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred_test)) / X_1_test.shape[0]
    accuracies.append(accuracy)
index = np.argmax(accuracies)
w = ws[index]

In [22]:
pred_tr = X_1_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_train - pred_tr)) / X_1_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

accuracy on training set is of 77.3264839726292%, loss is of 0.21451560123730073


In [23]:
pred = X_1_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred)) / X_1_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

accuracy on test set is of 77.40688919981345 %


## logistic regression

In [24]:
max_iter = 20000
gamma = 0.005
w, loss = logistic_regression(y_1_train, X_1_train, np.zeros(X_1_train.shape[1]), max_iter, gamma)

In [25]:
pred_tr = X_1_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_train - pred_tr)) / X_1_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

accuracy on training set is of 75.2077128598746%, loss is of 0.48639858062277364


In [26]:
pred = X_1_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred)) / X_1_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

accuracy on test set is of 75.40475714571258 %


## reg logistic regression

In [ ]:
max_iter = 20000
gamma = 0.005
lambda_ = 0
w, loss = reg_logistic_regression(y_1_train, X_1_train, lambda_, np.zeros(X_1_train.shape[1]), max_iter, gamma)

In [ ]:
pred_tr = X_1_train @ w
pred_tr = np.where(pred_tr > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_train - pred_tr)) / X_1_train.shape[0]
print("accuracy on training set is of {}%, loss is of {}".format(accuracy, loss))

In [ ]:
pred = X_1_test @ w
pred = np.where(pred > 1/2, 1, 0)
accuracy = 100 - 100 * np.sum(np.abs(y_1_test - pred)) / X_1_test.shape[0]
print("accuracy on test set is of {} %".format(accuracy))

# MLP

### backprop

For MSE:

$ 
    \frac{\delta L}{a_n} = \frac{\delta (a_n - y)^2}{\delta a_{n}} = 2(a_n - y)  \\
    \frac{\delta a_{i}}{\delta z_{i}} = \frac{\delta S(z_{i})}{\delta z_{i}} = S(z_{i})(1 - S(z_{i})) \\ 
    \frac{\delta z_{i+1}}{\delta w_{i}} = \frac{\delta (a_{i} * w_{i} + b_{i})}{\delta w_{i}} = a_{i} \\
    \frac{\delta z_{i+1}}{\delta b_{i}} = \frac{\delta (a_{i} * w_{i} + b_{i})}{\delta b_{i}} = 1  \\
    \frac{\delta z_{i+1}}{\delta a_{i}} = \frac{\delta (a_{i} * w_{i} + b_{i})}{\delta a_{i}}= w_{i} $

# tryin' ma best to vectorize baby

In [ ]:
def relu(z):
    return np.where(z < 0, 0, z)

def relu_gradient(z):
    return np.where(z < 0, 0, 1)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

def grad_loss(y_pred, y):
    return y_pred - y

def BCE_gradient(y,y_pred):
    #return y_pred-y
    return (-y/y_pred + (1-y)/(1-y_pred))

class layer:

    def __init__(self, dim_0, dim_1, activation):
        self.w = np.random.randn(dim_0, dim_1) / np.sqrt(dim_0)
        self.b = np.zeros(dim_0)
        if activation == 'relu':
            self.f = relu
            self.f_grad = relu_gradient
        if activation == 'sigmoid':
            self.f = sigmoid
            self.f_grad = sigmoid_gradient

    def feed_forward(self, a):
        self.a_prev = a
        if len(a.shape) == 1:
            # for batch_size 1
            self.z = np.dot(self.w, a) + self.b
        else:
            self.z = np.dot(self.w, a) + np.tile(self.b, (a.shape[1],1)).T
        self.a = self.f(self.z)
        return self.a

    def back_propagate(self, grad):
        grad = grad * self.f_grad(self.z)
        self.w_grad = grad @ self.a_prev.T
        self.b_grad = np.sum(grad, axis = 1)
        return (grad.T @ self.w).T


class MLP:
    # trying a vectorized MLP
    def __init__(self, dim, activations):
        
        self.layers = []
        for n in range(len(dim)-1):
            self.layers.append(layer(dim[n + 1], dim[n], activations[n]))
        
    def feed_forward(self, X):
        a = X
        for l in self.layers:
            a = l.feed_forward(a)
        return a
    
    def back_propagate(self, y_pred, y):
        grad = BCE_gradient(y, y_pred)
        for l in np.flip(self.layers):
            grad = l.back_propagate(grad)
        
            
    def gradient_descent_step(self, gamma, weight_decay):
        for l in self.layers:
            l.w -= (l.w_grad + l.w * weight_decay) * gamma
            l.b -= (l.b_grad + l.b * weight_decay) * gamma
            
    def train(self, X, Y, batch_size, max_iter, gamma, weight_decay, number_of_loss_computations = 5):
        
        start = datetime.now()
        gamma = gamma
        div = int(max_iter / number_of_loss_computations)
        
        for i in range(max_iter):
            
            if i == int(max_iter / 3):
                gamma = 0.1 * gamma
            if i == int(max_iter * 0.75):
                gamma = 0.1 * gamma
            if i % div == 0:
                print("{}% of the way".format(int(i/max_iter * 100)))
                print(self.BCE_loss(X, Y))
                
            idxs = np.arange(X.shape[0])
            np.random.shuffle(idxs)
            idxs = idxs[:batch_size]
            
            X_batch = X[idxs]
            y_batch = y[idxs]
            y_pred = self.feed_forward(X_batch.T)
            self.back_propagate(y_pred, y_batch)
            self.gradient_descent_step(gamma / batch_size, weight_decay)

        end = datetime.now()
        print("time taken:", end - start)
                
    def BCE_loss(self,X, y):
        loss = 0
        N = len(y)
        for i in range(N):
            y_pred = self.feed_forward(X[i])
            eps = 1e-7
            loss_i = -(y[i]*np.log(y_pred+eps) + (1-y[i])*np.log(1-y_pred+eps))
            loss = loss + loss_i/N
        
        return loss
    
    
    def BCE_loss_vect(self, X, y):
        y_pred = self.feed_forward(X.T)
        return np.mean(y_pred * np.log(y_pred + eps) + (1 - y_pred) * np.log(1 - y_pred + eps))

    
    def predict(self, X):
        y = self.feed_forward(X.T)
        return np.where(y < 0.5, 0, 1)

In [ ]:
np.random.seed(1)
in_dim = X_train.shape[1]
n_h1 = 100
n_h2 = 100
n_h3 = 100
n_h4 = 100
n_h5 = 100
n_h6 = 100
n_h7 = 100
out_dim = 1
dimensions = [in_dim, n_h1,n_h2,n_h3,n_h4,n_h5,n_h6,n_h7,out_dim]
activations = ['relu','relu','relu','relu','relu','relu','relu','sigmoid']

# mlp = MLP(gamma = gamma, dimensions = dimensions, activations = activations, weight_decay = weight_decay)

dimensions = [in_dim, n_h1,n_h2,n_h3,n_h4,n_h5,n_h6,n_h7,out_dim]
activations = ['relu','relu','relu','relu','relu','relu','relu','sigmoid']
mlp_1 = MLP(dimensions, activations)
dimensions = [in_dim, 30,30,30,out_dim]
activations = ['relu','relu','relu','sigmoid']
mlp_2 = MLP(dimensions, activations)
dimensions = [in_dim, 50,50,50,50,50,out_dim]
activations = ['relu','relu','relu','relu','relu','sigmoid']
mlp_3 = MLP(dimensions, activations)

In [ ]:
gamma = 0.001
weight_decay = 0.001
max_iter = 1750000
batch_size = 1

In [ ]:
mlp_1.train(X, y, batch_size, max_iter, gamma, weight_decay)

In [ ]:
y_pred1 = mlp_1.feed_forward(X_train.T)
acc_train_1 = 1-np.sum(np.abs(y_pred1 - y_train)) / X_train.shape[0]
y_pred1 = mlp_1.feed_forward(X_test.T)
acc_test_1 = 1-np.sum(np.abs(y_pred1 - y_test)) / X_test.shape[0]
print("first MLP")
print("training accuracy: {}% | test accuracy: {}%".format(acc_train_1 * 100, acc_test_1 * 100))

In [ ]:
mlp_2.train(X, y, batch_size, max_iter, gamma, weight_decay * 0.1)

In [ ]:
y_pred2 = mlp_2.feed_forward(X_train.T)
acc_train_2 = 1-np.sum(np.abs(y_pred2 - y_train)) / X_train.shape[0]
y_pred2 = mlp_1.feed_forward(X_test.T)
acc_test_2 = 1-np.sum(np.abs(y_pred2 - y_test)) / X_test.shape[0]
print("second MLP")
print("training accuracy: {}% | test accuracy: {}%".format(acc_train_2 * 100, acc_test_2 * 100))

In [ ]:
mlp_3.train(X, y, batch_size, max_iter, gamma, weight_decay * 0.1)

In [ ]:
y_pred3 = mlp_3.feed_forward(X_train.T)
acc_train_3 = 1-np.sum(np.abs(y_pred3 - y_train)) / X_train.shape[0]
y_pred3 = mlp_3.feed_forward(X_test.T)
acc_test_3 = 1-np.sum(np.abs(y_pred3 - y_test)) / X_test.shape[0]
print("third MLP")
print("training accuracy: {}% | test accuracy: {}%".format(acc_train_3 * 100, acc_test_3 * 100))

In [ ]:
_,X_sub,ids = load_csv_data("test.csv")
#feature 1: correlations der_mass_MMC
X_sub = np.where(X_sub == -999., np.nan, X_sub)
col_means = np.nanmean(X_sub, axis=0)
idxs = np.where(np.isnan(X_sub))
X_sub[idxs] = np.take(col_means, idxs[1])
X_gt_mmc = np.array(X_sub[:,0], copy=True)
X_gt_mmc[X_gt_mmc <= 140] = 140
# X = np.column_stack((X, X_gt_mmc))
X_sub[:,0][X_sub[:,0] > 140] = 140
X_sub = np.column_stack((X_sub, X_gt_mmc))

#feature 2: add momentums
#tau momentum
tau_px = X_sub[:,13]*np.cos(X_sub[:,15])
tau_py = X_sub[:,13]*np.sin(X_sub[:,15])
tau_pz = X_sub[:,13]*np.sinh(X_sub[:,14])
X_sub = np.column_stack((X_sub, tau_px,tau_py,tau_pz))
#lep momentum
lep_px = X_sub[:,16]*np.cos(X_sub[:,18])
lep_py = X_sub[:,16]*np.cos(X_sub[:,18])
lep_pz = X_sub[:,16]*np.cos(X_sub[:,17])
X_sub = np.column_stack((X_sub, lep_px,lep_py,lep_pz))
#leading jet momentum
jet_px = X_sub[:,22]*np.cos(X_sub[:,24])
jet_py = X_sub[:,22]*np.cos(X_sub[:,24])
jet_pz = X_sub[:,22]*np.cos(X_sub[:,23])
X_sub = np.column_stack((X_sub, jet_px,jet_py,jet_pz))
#subleading jet momentum
subjet_px = X_sub[:,25]*np.cos(X_sub[:,27])
subjet_py = X_sub[:,25]*np.cos(X_sub[:,27])
subjet_pz = X_sub[:,25]*np.cos(X_sub[:,26])
X_sub = np.column_stack((X_sub, subjet_px,subjet_py,subjet_pz))

# feature 3: abs angles
#der_met_phi_centrality
X_sub[:,11] = np.abs(X_sub[:,11])
#tau phi
X_sub[:,15] = np.abs(X_sub[:,15])
#lep phi
X_sub[:,18] = np.abs(X_sub[:,18])
#met phi
X_sub[:,20] = np.abs(X_sub[:,20])
#lead jet phi
X_sub[:,24] = np.abs(X_sub[:,24])
#sublead jet phi
X_sub[:,27] = np.abs(X_sub[:,27])

X_sub = make_features(X_sub)

In [ ]:
p_1 = mlp_1.feed_forward(X_sub.T)
p_2 = mlp_2.feed_forward(X_sub.T)
p_3 = mlp_3.feed_forward(X_sub.T)
p = np.mean((p_1,p_2,p_3),axis = 0)
p = p > 0.5
sub_pred = p*2 -1

In [ ]:
sub_pred = sub_pred.squeeze()

In [ ]:
create_csv_submission(ids, sub_pred, "submission_test.csv")

In [ ]:
t = pd.read_csv("submission_test.csv")

In [ ]:
t.to_csv("submission_test.csv",index = False)

In [ ]:
import numpy as np
from implementations import *
from proj1_helpers import *

y,X,ids = load_csv_data("train.csv")

X = np.where(X == -999., np.nan, X)
X_ = X

import pandas as pd
mass_nan_index = [0]
jet_nan_indexes = [4,5,6,12,26,27,28]
jet_sub_nan_indexes = [23,24,25]

### masses prediction

In [ ]:
X_noNans = np.delete(X_, np.concatenate((jet_nan_indexes, jet_sub_nan_indexes)), 1)

defined_mass_idx = np.where(np.isfinite(X_noNans[:,0])) #indexes of samples with a mass
undefined_mass_idx = np.where(np.isnan(X_noNans[:,0]))  #indexes of samples with a missing mass

X_mass = X_noNans[defined_mass_idx][:,1:] # we select the X of training set
y_mass = X_noNans[defined_mass_idx][:,:1] # we select the y of the taining set

X_mass,_,_ = standardize(X_mass) # we standardize both 
y_mass,_,_ = standardize(y_mass)
y_mass = y_mass.squeeze()

In [ ]:
initial_w = np.ones(X_mass.shape[1]) / X_mass.shape[1]
lambda_ = 0.0001
max_iter = 1000
gamma = 0.005
w, loss = least_squares_GD(y_mass, X_mass, initial_w, max_iter, gamma) # since it isn't classification, we use least_squares

In [ ]:
to_predict_mass,_,_ = standardize(X_noNans[undefined_mass_idx][:,1:]) # we select the set to be predicted
predicted_masses = to_predict_mass @ w

In [ ]:
masses = np.zeros(X_.shape[0])
masses[defined_mass_idx] = y_mass
masses[undefined_mass_idx] = predicted_masses

In [ ]:
X_[:,mass_nan_index[0]] = masses

### jet predictions

In [ ]:
X_noNans = np.delete(X_, np.concatenate((mass_nan_index, jet_sub_nan_indexes)), 1)

defined_jet_idx = np.where(np.isfinite(X_[:,jet_nan_indexes[0]]))[0] #indexes of samples with jets
undefined_jet_idx = np.where(np.isnan(X_[:,jet_nan_indexes[0]]))[0]  #indexes of samples with missing jets

X_jet = np.delete(X_, np.concatenate((mass_nan_index, jet_nan_indexes, jet_sub_nan_indexes)), 1)[defined_jet_idx] # we select the X of training set
to_predict_jets = np.delete(X_, np.concatenate((mass_nan_index, jet_nan_indexes, jet_sub_nan_indexes)), 1)[undefined_jet_idx]
y_jet_1 = X_[defined_jet_idx][:,jet_nan_indexes[0]] # we select the y of the taining set [4]
y_jet_2 = X_[defined_jet_idx][:,jet_nan_indexes[1]] # we select the y of the taining set [5]
y_jet_3 = X_[defined_jet_idx][:,jet_nan_indexes[2]] # we select the y of the taining set [6]
y_jet_4 = X_[defined_jet_idx][:,jet_nan_indexes[3]] # we select the y of the taining set [12]
y_jet_5 = X_[defined_jet_idx][:,jet_nan_indexes[4]] # we select the y of the taining set [26]
y_jet_6 = X_[defined_jet_idx][:,jet_nan_indexes[5]] # we select the y of the taining set [27]
y_jet_7 = X_[defined_jet_idx][:,jet_nan_indexes[6]] # we select the y of the taining set [28]


In [ ]:
X_jet,_,_ = standardize(X_jet) # we standardize
to_predict_jets,_,_ = standardize(to_predict_jets)
y_jet_1,_,_ = standardize(y_jet_1)
y_jet_2,_,_ = standardize(y_jet_2)
y_jet_3,_,_ = standardize(y_jet_3)
y_jet_4,_,_ = standardize(y_jet_4)
y_jet_5,_,_ = standardize(y_jet_5)
y_jet_6,_,_ = standardize(y_jet_6)
y_jet_7,_,_ = standardize(y_jet_7)

y_jet_1 = y_jet_1.squeeze()
y_jet_2 = y_jet_2.squeeze()
y_jet_3 = y_jet_3.squeeze()
y_jet_4 = y_jet_4.squeeze()
y_jet_5 = y_jet_5.squeeze()
y_jet_6 = y_jet_6.squeeze()
y_jet_7 = y_jet_7.squeeze()

In [ ]:
y_jets = [y_jet_1, y_jet_2, y_jet_3, y_jet_4, y_jet_4, y_jet_5, y_jet_6, y_jet_7]

In [ ]:
lambda_ = 0.0001
max_iter = 1000
gamma = 0.005
initial_w = np.ones(X_jet.shape[1]) / X_jet.shape[1]

ws = []
losses = []

for y_jet in y_jets:
    w, loss = least_squares_GD(y_jet, X_jet, initial_w, max_iter, gamma) # since it isn't classification, we use least_squares
    ws.append(w)
    losses.append(loss)

In [ ]:
jet_predictions = []
for w in ws:
    jet_prediction = to_predict_jets @ w
    jet_predictions.append(jet_prediction)

In [ ]:
for i in range(len(jet_nan_indexes)):
    jet_col = np.zeros(X_.shape[0])
    jet_col[defined_jet_idx] = y_jets[i]
    jet_col[undefined_jet_idx] = jet_predictions[i]
    X_[:, jet_nan_indexes[i]] = jet_col